In [1]:
import tqdm
# ^^^ pyforest auto-imports - don't write above this line
import numpy as np
import pandas as pd

from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score, normalized_mutual_info_score
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.cluster import SpectralClustering

from sklearn.decomposition import PCA, SparsePCA, KernelPCA
from sklearn.manifold import TSNE

from rpy2.robjects import r, pandas2ri
from rpy2.robjects.vectors import StrVector

pandas2ri.activate()

import magic
import scprep

%matplotlib inline

# from sklearnex import patch_sklearn
# patch_sklearn()

import warnings

from sklearn.cluster import KMeans
from tqdm import tqdm

r('''
    source('../../imputation2/notebooks/repos/ALRA/alra.R')
''')

value,[RTYPES.CLOSXP]
visible,[RTYPES.LGLSXP]


In [2]:
def get_cluster_metrics(pred, labels):
    ari_res = []
    ami_res = []
    nmi_res = []

#     try:
#         r.assign("data", pred.T)
#         seurat = r('''
#             countsData = data.frame(data)
#             pbmc <- CreateSeuratObject(counts = countsData, project = "thal_single_cell", min.cells = 1, min.features = 1)
#             pbmc <- FindVariableFeatures(pbmc, selection.method = "vst", verbose=FALSE)
#             all.genes <- rownames(pbmc)
#             pbmc <- ScaleData(pbmc, features = all.genes, verbose=FALSE)
#             pbmc <- RunPCA(pbmc, verbose=FALSE)
#             pbmc <- FindNeighbors(pbmc, verbose=FALSE)
#             pbmc <- FindClusters(pbmc, verbose=FALSE)
#             Idents(pbmc)
#         ''')
#         ari_res.append(adjusted_rand_score(labels, seurat))
#         ami_res.append(adjusted_mutual_info_score(labels, seurat))
#         nmi_res.append(normalized_mutual_info_score(labels, seurat))
#     except Exception as e:
#         pass

    pred_ = KMeans(n_clusters=len(np.unique(labels)), random_state=42).fit_predict(pred)

    ari_res.append(adjusted_rand_score(labels, pred_))
    ami_res.append(adjusted_mutual_info_score(labels, pred_))
    nmi_res.append(normalized_mutual_info_score(labels, pred_))

    warnings.filterwarnings("error")

    affinities = ['cosine', 'linear', 'poly']

    for i in affinities:
        try:
            pred_ = SpectralClustering(
                n_clusters=len(np.unique(labels)), 
                random_state=42, 
                affinity=i
            ).fit_predict(pred)
            ari_res.append(adjusted_rand_score(labels, pred_))
            ami_res.append(adjusted_mutual_info_score(labels, pred_))
            nmi_res.append(normalized_mutual_info_score(labels, pred_))
        except:
            ari_res.append(0)
            ami_res.append(0)
            nmi_res.append(0)

    warnings.resetwarnings()
    
    return max(ari_res), max(ami_res), max(nmi_res)

In [3]:
def get_data(i):
    df = pd.read_csv('../data/{}/data.csv.gz'.format(i), index_col=0)
    tmp = np.sign(df)
    cols = (np.sum(tmp) > int((df.shape[0])*0.05))
    rows = (np.sum(tmp, axis=1) > int((df.shape[1])*0.05))
    df = np.log(df.loc[rows, cols] + 1)
    df_norm = df.copy()
    df_norm = scprep.normalize.library_size_normalize(df_norm)    
    df_norm = scprep.transform.sqrt(df_norm)
    X_norm = pd.DataFrame(df_norm, columns=df.columns)
    labels = df.index
    return X_norm, labels

In [4]:
dir_list = !ls ../data/ | grep -v zip
dir_list

['baron',
 'bmcite',
 'brosens',
 'carey',
 'cbmc',
 'chang',
 'Fujii',
 'hcabm40k',
 'hrvatin',
 'jakel',
 'jiang',
 'loureiro',
 'manno',
 'mingyao',
 'pbmc3k',
 'Selewa',
 'Xu']

In [5]:
len(dir_list)

17

In [6]:
res = []
for i in tqdm(dir_list):
    X_norm, labels = get_data(i)
    r.assign("A", X_norm)
    r('''
        A = as.matrix(A)
    ''')
    r('''
        # Library and log normalize the data
        A_norm <- normalize_data(A)
        # Choose k. 
        k_choice <- choose_k(A_norm)
    ''')
    pred = r('''
        A_norm_completed <- alra(A_norm,k=k_choice$k)[[3]]
    ''')

    res.append(get_cluster_metrics(pred, labels))

  0%|          | 0/17 [00:00<?, ?it/s]

Read matrix with 1791 cells and 8669 genes
Getting nonzeros
Randomized SVD
Find the 0.001000 quantile of each gene
Sweep
Scaling all except for 0 columns
0.00% of the values became negative in the scaling process and were set to zero
The matrix went from 21.79% nonzero to 77.80% nonzero


  6%|▌         | 1/17 [01:08<18:10, 68.15s/it]

Read matrix with 5445 cells and 5421 genes
Getting nonzeros
Randomized SVD
Find the 0.001000 quantile of each gene
Sweep
Scaling all except for 0 columns
0.00% of the values became negative in the scaling process and were set to zero
The matrix went from 23.76% nonzero to 76.38% nonzero


 12%|█▏        | 2/17 [03:50<30:57, 123.80s/it]

...

 88%|████████▊ | 15/17 [32:29<03:46, 113.38s/it]

Read matrix with 479 cells and 4021 genes
Getting nonzeros
Randomized SVD
Find the 0.001000 quantile of each gene
Sweep
Scaling all except for 0 columns
0.00% of the values became negative in the scaling process and were set to zero
The matrix went from 25.36% nonzero to 85.02% nonzero


 94%|█████████▍| 16/17 [33:26<01:36, 96.25s/it] 

Read matrix with 4252 cells and 7389 genes
Getting nonzeros
Randomized SVD
Find the 0.001000 quantile of each gene
Sweep
Scaling all except for 0 columns
0.00% of the values became negative in the scaling process and were set to zero
The matrix went from 26.24% nonzero to 71.87% nonzero


100%|██████████| 17/17 [36:40<00:00, 129.46s/it]


In [8]:
pd.DataFrame(res, columns=['ARI', 'AMI', 'NMI'], index=dir_list)

,ARI,AMI,NMI
baron,0.423489,0.714292,0.720415
bmcite,0.602631,0.693220,0.693521
brosens,0.496716,0.629648,0.632098
carey,0.651048,0.708789,0.709792
cbmc,0.469919,0.643651,0.650052
chang,0.135111,0.255810,0.279506
Fujii,0.346217,0.474879,0.478305
hcabm40k,0.089482,0.131142,0.133736
hrvatin,0.602051,0.797942,0.805445
jakel,0.441871,0.598558,0.600341
